## We want to align an external video with the LSL-synced audio of the same event, then cut trial-sized segments based on LSL timing information. Finally, overlay the audio of each trial to their corresponding cut videos to check synchronization.

In [ ]:
# IMPORTS
import os
import numpy as np
import pandas as pd
import sys
from matplotlib import pyplot as plt
import librosa
import subprocess
from IPython.display import Audio
import shign
from shign.shign import ms_to_samples
from scipy.io.wavfile import write

In [ ]:
# File paths:
video_file = "external_video.mp4" # Video to be synced
csv_file = "lsl_synced_long_audio_raw.csv" # LSL synced audio in raw format

# Output Names:
extracted_video_audio = "extracted_video_audio.wav"
lsl_synced_audio = "lsl_synced_audio.wav"
aligned_video_audio = "aligned_video_audio.wav"

## Inspecting the External Video for Sample Rate Information

In [ ]:
# Inspecting the video:
result = subprocess.run(
    ["ffmpeg",
     "-hide_banner",
     "-i", video_file],
    capture_output=True,
    text=True
)
print(result.stderr)

## Extracting the Audio from Video

In [ ]:
# Extracting the audio from the video:
subprocess.run([
    "ffmpeg",              
    "-i", video_file,      
    "-vn",                      # Exclude the video stream
    "-acodec", "pcm_s16le",
    "-ar", "44100",             # Audio sample rate: 44100 Hz (We checked this above!)
    "-ac", "1",                 # Converting the stereo audio to mono for consistency with the lsl_synced_audio
    extracted_video_audio       
], check=True)

## Converting Raw LSL Audio in CSV File to WAV

In [ ]:
# Saving raw lsl_synced_audio to .wav format:
audio_data = pd.read_csv(csv_file, header=None)
amplitude = audio_data[1].values.astype(np.int16)
sample_rate = 16000                                     # Sample rate was 16 kHz
write(lsl_synced_audio, sample_rate, amplitude)

## Align the Two Audio Tracks Using "shign"

In [ ]:
lsl_synced_audio, sr_lsl = librosa.load("lsl_synced_audio.wav", sr=None)
extracted_audio, sr_ext = librosa.load("extracted_video_audio.wav", sr=None)
# Downsampling is a necessary step:
extracted_audio_downsampled = librosa.resample(extracted_audio, orig_sr=sr_ext, target_sr=sr_lsl)
sr_ext_downsampled = sr_lsl

In [ ]:
# Plotting two audio before synchronization:
plt.plot(lsl_synced_audio, label='lsl_audio')
plt.plot(extracted_audio_downsampled, label='extracted_audio')
plt.legend(loc='lower left')
plt.show()

In [ ]:
_, extracted_audio_aligned, shift_ms = shign.shift_align(      # Saving shift_ms here to use it later
    audio_a = lsl_synced_audio, 
    audio_b = extracted_audio_downsampled, 
    sr_a    = sr_lsl, 
    sr_b    = sr_ext_downsampled, 
    align_how = "pad_and_crop_one_to_match_other",      # Do not modify the lsl_audio
    max_shift_sec = 300
)

In [ ]:
print(f"Mismatch between two audio is {shift_ms/1000} seconds.")

# Negative means the second audio starts later.

In [ ]:
plt.plot(lsl_synced_audio, label='lsl_audio')
plt.plot(extracted_audio_aligned, label='extracted_audio_aligned')
plt.legend(loc='lower left')
plt.show()

In [ ]:
print(len(lsl_synced_audio))
Audio(lsl_synced_audio, rate=sr_lsl)

In [ ]:
print(len(extracted_audio_aligned))
Audio(extracted_audio_aligned, rate=sr_ext_downsampled)

In [ ]:
# Saving the extracted_audio_aligned:
write("extracted_audio_aligned.wav", sr_ext_downsampled, extracted_audio_aligned)

## Aligning the External Video by Trimming (based on the computed "shift_ms" calculated from extracted audio)

In [ ]:
# Paths
video_file = "external_video.mp4"
output_video = "external_video_aligned.mp4"

In [ ]:
start_sample = ms_to_samples(abs(shift_ms), sr=sr_ext_downsampled)
print(f"extracted_audio_aligned starts at: {abs(shift_ms):.2f} milliseconds")
print(f"extracted_audio_aligned starts at sample {start_sample:.2f}")

In [ ]:
aligned_length_samples = len(extracted_audio_aligned)
end_sample = start_sample + aligned_length_samples
print(f"extracted_audio_aligned ends at sample {end_sample:.2f}")

In [ ]:
# Calculating the video cut times as seconds, using start and end samples and sample rate:
start_time_sec = start_sample / sr_ext_downsampled
end_time_sec = end_sample / sr_ext_downsampled
print(f"Aligned video starts at {start_time_sec:.2f}th second")
print(f"Aligned video ends at {end_time_sec:.2f}th second")

In [ ]:
# Cut the external_video_aligned using ffmpeg:
print(f"Trimming video from {start_time_sec:.2f}s to {end_time_sec:.2f}s...")
print(f"Video length is {end_time_sec-start_time_sec:.2f} seconds...")
subprocess.run([
    "ffmpeg",
    "-ss", f"{start_time_sec:.3f}",    # Start time BEFORE -i ensures frame accuracy
    "-i", video_file,                  
    "-to", f"{end_time_sec - start_time_sec:.3f}",  # Duration after start
    "-c:v", "libx264",                 # Re-encode video for frame accuracy
    "-c:a", "aac",                     
    "-preset", "fast",                 # Encoding speed optimization
    "-reset_timestamps", "1",          # Reset timestamps after cutting
    output_video                       
], check=True)

print(f"Aligned video saved as {output_video}")

## Computing the Trial Times from Each Trial's LSL-Synced Raw Audio in CSV Files

In [ ]:
# Folder containing the LSL synced audio .csv files of all trials:
csv_folder = "CSV_Files"

results = []

# Iterate over all CSV files in the folder:
for file_name in os.listdir(csv_folder):
    if file_name.endswith(".csv"):
        file_path = os.path.join(csv_folder, file_name)
            
        # For each file:
        try:
            data = pd.read_csv(file_path, header=None, names=["time_ms", "amplitude"])
                
            # Get the start and end times
            start_time = data["time_ms"].iloc[1]
            end_time = data["time_ms"].iloc[-1]
                
            # Append the results to the list, including the filename
            results.append({
                "file_name": file_name,
                "start_time_ms": start_time,
                "end_time_ms": end_time
            })
        except Exception as e:
            print(f"Error processing file {file_name}: {e}")
    
# Convert results to DataFrame:
results_df = pd.DataFrame(results)

# Save the DataFrame:
output_filename = "trial_times.csv"
results_df.to_csv(output_filename, index=False)

In [ ]:
# Printing the first 10 rows to inspect the accuracy of the times:
results_df.head(10)

## Mapping the Global Trial Start and End Times to Video Start and End Times

In [ ]:
# Parameters and inputs
lsl_audio_sr = sr_lsl
extracted_audio_sr = sr_ext_downsampled
csv_file = "0_1_Mic_nominal_srate16000.csv"  # raw lsl_synced_audio as a .csv file
trial_times_file = "trial_times.csv" # previously saved trial start and end times
mapped_output_file = "mapped_event_markers.csv" # output name

In [ ]:
# Load files
lsl_audio_raw = pd.read_csv(csv_file, header=None, names=["time_ms", "value"])
trial_times = pd.read_csv(trial_times_file)

In [ ]:
# Add new columns for the results
trial_times["lsl_sample_start"] = None  # Sample number in lsl stream for start
trial_times["lsl_sample_end"] = None  # Sample number in lsl stream for end
trial_times["video_time_start"] = None  # Time in video (seconds) for start
trial_times["video_time_end"] = None  # Time in video (seconds) for end

In [ ]:
# Iterate over all rows
for idx, row in trial_times.iterrows():
    # Calculate lsl_sample_start by counting rows up to start_time_ms
    lsl_sample_start = lsl_audio_raw[lsl_audio_raw["time_ms"] <= row["start_time_ms"]].shape[0]
    
    # Calculate lsl_sample_end by counting rows up to end_time_ms, similarly
    lsl_sample_end = lsl_audio_raw[lsl_audio_raw["time_ms"] <= row["end_time_ms"]].shape[0]

    # Calculate video start and end times based on sample rate (in seconds)
    video_time_start = lsl_sample_start / lsl_audio_sr
    video_time_end = lsl_sample_end / lsl_audio_sr

    # Save results
    trial_times.at[idx, "lsl_sample_start"] = lsl_sample_start
    trial_times.at[idx, "lsl_sample_end"] = lsl_sample_end
    trial_times.at[idx, "video_time_start"] = round(video_time_start, 6)
    trial_times.at[idx, "video_time_end"] = round(video_time_end, 6)

In [ ]:
trial_times.head(10)

In [ ]:
# Save the DataFrame in a different file:
trial_times.to_csv(mapped_output_file, index=False)

## Cutting the Trial Videos

In [ ]:
# Load the data including the time markers to cut the aligned video:
markers = pd.read_csv("mapped_event_markers.csv")

# Input video:
input_video = "external_video_aligned.mp4"

# Output folder:
output_folder = "cut_videos/"

In [ ]:
os.makedirs(output_folder, exist_ok=True)

for i, row in markers.iterrows():
    start_time = row["video_time_start"]
    end_time = row["video_time_end"]
    file_name = row["file_name"]
    
    # Create an output file name based on the input filename (.mp4 instead of .csv):
    output_file = os.path.join(output_folder, f"{file_name.replace('.csv', '')}.mp4")
    
    # Use FFmpeg to cut the video with accurate timestamps and no audio
    ffmpeg_command = [
    "ffmpeg",
    "-ss", f"{start_time:.3f}",         
    "-i", input_video,                  
    "-to", f"{end_time - start_time:.3f}", 
    "-c:v", "libx264",                  
    "-an",                              # Disable audio (for trial videos)
    "-preset", "fast",                  # Faster encoding
    "-reset_timestamps", "1",           # Reset timestamps globally
    "-filter_complex", 
    "[0:v]setpts=PTS-STARTPTS[v]",      # Reset video PTS
    "-map", "[v]",                      # Map video stream
    "-movflags", "+faststart",          # Optimize for playback
    output_file
    ]
    
    # Execute the command using subprocess:
    subprocess.run(ffmpeg_command, check=True)
    print(f"Segment saved: {output_file}")

## (Optional) Overlaying the Audio of Each Trial to Their Cut Videos

In [ ]:
# Paths
current_folder = os.getcwd()
video_folder = "./cut_videos" # folder including all the cut trial videos
audio_folder = "./audio_files" # folder including all the trial audio files (names should match)
output_folder = os.path.join(current_folder, "audio_overlay") # Output folder

# Create the output folder if it doesn't exist:
os.makedirs(output_folder, exist_ok=True)

# Get sorted lists of video and audio files (only works if the names of the audio files and video files match)
video_files = sorted([f for f in os.listdir(video_folder) if f.endswith(".mp4")])
audio_files = sorted([f for f in os.listdir(audio_folder) if f.endswith(".wav")])

for video_file, audio_file in zip(video_files, audio_files):
    # Check if the first 10 characters of filenames match (as a second confirmation)
    if video_file[:10] == audio_file[:10]:
        video_path = os.path.join(video_folder, video_file)
        audio_path = os.path.join(audio_folder, audio_file)
        # Save with the same name as the original .mp4 file in the output folder
        output_path = os.path.join(output_folder, video_file)
        
        ffmpeg_command = [
            "ffmpeg",
            "-i", video_path,    
            "-i", audio_path,     
            "-c:v", "copy",       
            "-c:a", "aac",        
            "-map", "0:v:0",      # Map the first input's video
            "-map", "1:a:0",      # Map the second input's audio
            output_path           
        ]
        
        # Run the FFmpeg command
        print(f"Overlaying audio for: {video_file} with {audio_file}")
        subprocess.run(ffmpeg_command)
        print(f"Saved with overlay audio: {output_path}")
    else:
        print(f"No matching audio found for video: {video_file}")
print("All videos processed!")